<hr style="border:2px solid blue; margin-top:0;">


![Vintage Movie Cassette Tape](https://media.discordapp.net/attachments/1008571220385599488/1126232701146501360/MadsMeyer_peaky_blinders_style_painting_2e662a64-2009-46fd-b8bf-12e84648efc4.png?width=573&height=573)

In [1]:
import sqlite3
import pandas as pd

# Define a dictionary of file paths and corresponding DataFrame names
files = {
    'bom_movie_gross': 'bom.movie_gross.csv.gz',
    'rt_movie_info': 'rt.movie_info.tsv.gz',
    'rt_reviews': 'rt.reviews.tsv.gz',
    'tmdb_movies': 'tmdb.movies.csv.gz',
    'tn_movie_budgets': 'tn.movie_budgets.csv.gz'
}

# Load all the files into DataFrames in a single line of code
df_dict = {name: pd.read_csv(path, sep='\t' if path.endswith('.tsv.gz') else ',', encoding='ISO-8859-1') for name, path in files.items()}


# Verify successful loading by checking the DataFrame names
df_dict.keys()


dict_keys(['bom_movie_gross', 'rt_movie_info', 'rt_reviews', 'tmdb_movies', 'tn_movie_budgets'])

In [2]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('movies.db')

# Write the dataframes to SQLite tables
for table_name, df in df_dict.items():
    df.to_sql(table_name, conn, if_exists='replace', index=False)

# Verify the tables in the SQLite database
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# Close the database connection
conn.close()

# Extract table names
table_names = [table[0] for table in tables]

print(table_names)

['bom_movie_gross', 'rt_movie_info', 'rt_reviews', 'tmdb_movies', 'tn_movie_budgets']


C:\Users\HP\anaconda3\envs\learn-env\lib\site-packages\pandas\core\generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [3]:
# Connect to the SQLite database
conn = sqlite3.connect('movies.db')

# Delete the 'bom_movie_gross_df' table
conn.execute("DROP TABLE IF EXISTS bom_movie_gross_df")

# Verify the tables in the SQLite database
tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

# Close the database connection
conn.close()

# Extract table names
table_names = [table[0] for table in tables]

table_names




['bom_movie_gross',
 'rt_movie_info',
 'rt_reviews',
 'tmdb_movies',
 'tn_movie_budgets']

# Title: Microsoft-Movie-Studio-Box-Office-Analysis
***
**Author**: [Oscar Mulei](mailto:omulei@gmail.com)
***

## Overview
***
This project provides a comprehensive analysis of box office performance for movies. The goal is to offer valuable insights to Microsoft's newly established movie studio. The analysis covers various aspects of movies, such as genres, ratings, budgets, and release dates, to uncover the factors that contribute to a successful box office performance.
***

## Business Problem
***
![img](https://media.discordapp.net/attachments/1050435586235846767/1052697908295774298/yassein_muhamedddd_action_movie_poster_movie_poster_action_movi_5d7ba72b-db05-4212-917c-85855d4425c1.png?width=573&height=573)

Microsoft, recognizing the success of other big companies in creating original video content, is eager to enter the movie industry. However, despite their eagerness, they lack experience in movie creation. The challenge lies in understanding the current movie landscape and identifying the types of films that perform well at the box office. This project aims to explore these aspects, and its findings will guide the decision-making process for Microsoft's new movie studio, helping to decide what type of films to create.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data
***
The project leverages several datasets, providing comprehensive movie-related information. Each movie has a unique ID associated with its [intake](link-to-intake-dataset) and [outcome](link-to-outcome-dataset) data. The datasets offer valuable data points, such as movie titles, genres, average ratings, domestic and foreign gross revenues, and release years.

Given the business problem, we recognize the need to incorporate additional data to provide more comprehensive insights. This includes information on movie genre, budget and box office revenue, director and cast, runtime, and regional and language specifics. However, our current dataset includes the movie name, release year, and IMDB rating. Future enhancements of this analysis may include these additional data points.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

In [4]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [5]:
# Here you run your code to explore the data

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [6]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [7]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***